In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
from IPython.display import clear_output

In [12]:
response = requests.get("http://emart.ssg.com/")
bs = BeautifulSoup(response.content, "html.parser")

category = bs.select('li.em_total_mn a')

category_ls = []

for i in category:
    category_name = i.text
    category_number = str(i.attrs["href"])[32:42]
    category_ls.append(category_number)


category_dict = defaultdict(str)

for i in range(len(category_ls)):
    response = requests.get("http://emart.ssg.com/category/listCategoryItem.ssg?dispCtgId={}".format(category_ls[i]))
    bs = BeautifulSoup(response.content, "html.parser")

    category2 = bs.select('li.none_child a')

    for j in category2:
        category_dict[j.text] = j.attrs["data-ilparam-value"]
        
    
    clear_output(wait=True)
    print('{}% 진행중...'.format(int(i/281*100)))

99% 진행중...


In [15]:
category_dict

defaultdict(str,
            {'사과': '6000095799',
             '배': '6000095800',
             '감귤': '6000095814',
             '만감류': '6000095815',
             '수박': '6000095829',
             '멜론': '6000095830',
             '참외': '6000095831',
             '방울토마토': '6000095832',
             '토마토': '6000095833',
             '딸기': '6000095844',
             '키위': '6000095845',
             '블루베리': '6000095846',
             '포도': '6000095864',
             '자두': '6000095866',
             '복숭아': '6000095868',
             '단감/홍시': '6000095889',
             '곶감/반건시': '6000095890',
             '바나나': '6000095904',
             '파인애플': '6000095905',
             '오렌지': '6000095919',
             '자몽': '6000095920',
             '레몬': '6000095921',
             '기타 시트러스': '6000095922',
             '망고': '6000095937',
             '기타 열대과일': '6000095940',
             '체리': '6000095964',
             '석류': '6000095965',
             '아보카도': '6000095966',
             '건과일': '60000959